<a href="https://colab.research.google.com/github/guimasuko/project/blob/main/Aula01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
!pip install yfinance --upgrade --no-cache-dir
!pip install investpy

In [4]:
import yfinance as yf
import investpy as inv

In [5]:
import time
import datetime

### Taxa de juros Brasil (Selic)

Códigos para obtenção de dados do bc:

https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries

In [6]:
# função que retorna dados do banco central através de códigos
def consulta_bc(codigo_bcb):
  url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bcb)
  df = pd.read_json(url)
  df['data'] = pd.to_datetime(df['data'], dayfirst=True)
  df.set_index('data', inplace=True)
  return df

In [7]:
# código da meta da taxa de juros brasileira (selic)
selic = consulta_bc(432)

In [8]:
selic.rename(columns={'valor': 'Selic'}, inplace=True)
selic.head()

,Selic
data,
1999-03-05,45.0
1999-03-06,45.0
1999-03-07,45.0
1999-03-08,45.0
1999-03-09,45.0


### Dólar e Ibovespa

In [10]:
tickers = "^BVSP USDBRL=X"
dollar_ibov = yf.download(tickers, start="2007-01-01")["Close"]

In [ ]:
dollar_ibov.rename(columns={'USDBRL=X': 'Dollar', '^BVSP': 'Ibovespa'}, inplace=True)

In [ ]:
dollar_ibov.head()

,Dollar,Ibovespa
Date,,
2007-01-01,2.1325,NaN
2007-01-02,2.1310,45383.0
2007-01-03,2.1345,44445.0
2007-01-04,2.1390,44020.0
2007-01-05,2.1470,42245.0


### Unindo os dois dataframes.

In [ ]:
df = pd.merge(dollar_ibov, selic, left_index=True, right_index=True)
df

,Dollar,Ibovespa,Selic
2007-01-01,2.132500,NaN,13.25
2007-01-02,2.131000,45383.00000,13.25
2007-01-03,2.134500,44445.00000,13.25
2007-01-04,2.139000,44020.00000,13.25
2007-01-05,2.147000,42245.00000,13.25
...,...,...,...
2022-02-09,5.258400,112461.00000,10.75
2022-02-10,5.234500,113359.00000,10.75
2022-02-11,5.248700,113495.00000,10.75
2022-02-14,5.249956,113807.00000,10.75
